# 26/04/2023

### Importar las librerías

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
import sqlite3
from datetime import datetime
import schedule
import time
import csv


## Web Scraping con Selenium
### Conexión con el driver

In [5]:
# Opciones de navegacion

options =  webdriver.ChromeOptions()
options.add_argument('--start-maximized')
options.add_argument('--disable-extensions')

driver_path = Service("C:\\Users\\Tatu\\Downloads\\chromedriver_win32\\chromedriver.exe")

driver = webdriver.Chrome(service = driver_path, options = options)


In [6]:
# Conexion con la pagina
driver.get('https://es.investing.com/equities/ypf-sociedad')

tit=driver.title
print(tit)


Acciones YPF | Cotización BA:YPFD hoy - Investing.com


In [4]:
# Insertar fecha y hora actual
now = datetime.now()
print(now)

2023-05-08 14:36:25.905009


### Obtención de los datos
Se debe actualizar el fullXPath, no se mantiene siempre igual

In [ ]:
# Obtencion de los datos con el fullXPath

ultimo_cierre = driver.find_element(By.XPATH, '/html/body/div[1]/div[2]/div[2]/div/div[1]/div/div[5]/div[1]/dl[1]/div[1]/dd/span/span[1]')
ultimo_cierre = ultimo_cierre.text
print("Último cierre " + ultimo_cierre)

rango_min = driver.find_element(By.XPATH, '/html/body/div/div[2]/div[2]/div/div[1]/div/div[5]/div[1]/dl[1]/div[3]/dd/span[1]/span[1]')
rango_min = rango_min.text
print("Mínimo " + rango_min)

rango_max = driver.find_element(By.XPATH, '/html/body/div/div[2]/div[2]/div/div[1]/div/div[5]/div[1]/dl[1]/div[3]/dd/span[3]/span[1]')
rango_max = rango_max.text
print("Máximo " + rango_max)

ingresos_T = driver.find_element(By.XPATH, '/html/body/div/div[2]/div[2]/div/div[1]/div/div[5]/div[1]/dl[2]/div[2]/dd/span/span[1]')
ingresos_T = ingresos_T.text
print("Ingresos " + ingresos_T + "T")

apertura = driver.find_element(By.XPATH, '/html/body/div/div[2]/div[2]/div/div[1]/div/div[5]/div[1]/dl[1]/div[2]/dd/span/span[1]')
apertura = apertura.text
print("Apertura " + apertura)


volumen = driver.find_element(By.XPATH, '/html/body/div/div[2]/div[2]/div/div[1]/div/div[5]/div[1]/dl[1]/div[5]/dd/span/span[1]')
volumen = volumen.text
print("Volumen " + volumen)

volumen_promedio = driver.find_element(By.XPATH, '/html/body/div/div[2]/div[2]/div/div[1]/div/div[5]/div[1]/dl[1]/div[6]/dd/span/span[1]')
volumen_promedio = volumen_promedio.text
print("Volumen promedio " + volumen_promedio)

ultimo_precio = driver.find_element(By.XPATH, '/html/body/div/div[2]/div[2]/div/div[1]/div/div[1]/div[3]/div/div[1]/div[1]')
ultimo_precio = ultimo_precio.text
print("Último precio " + ultimo_precio)

In [7]:
# Cerrar el driver
driver.close()

## Guardar los datos obtenidos en una base de datos SQL
### Conexión con sqlite3

In [3]:
#Conexión con la base de datos
conn = sqlite3.connect('acciones_YPFD.db')
c = conn.cursor()

In [ ]:
#Creación de la tabla "resumen"
#c.execute('''CREATE TABLE resumen
#             (date_time DATETIME, ultimo_precio REAL, minimo REAL, maximo REAL, ingresos REAL, apertura REAL, volumen REAL, volumen_promedio REAL)''')

In [9]:
# Insertar datos en la tabla
c.execute("INSERT INTO resumen (date_time, ultimo_cierre, minimo, maximo, ingresos, apertura, volumen, volumen_promedio, ultimo_precio) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)", 
          (now, ultimo_cierre, rango_min, rango_max, ingresos_T, apertura, volumen, volumen_promedio, ultimo_precio))

# Guardar los cambios en la base de datos y cerrar la conexión
conn.commit()


In [10]:
# Ver los datos guardados
# Seleccionar los datos de la tabla
c.execute("SELECT * FROM resumen")

# Obtener los resultados de la consulta
resultados = c.fetchall()

# Imprimir los resultados en la consola
for fila in resultados:
    print(fila)

('2023-05-05 23:11:14.474165', '4.768,75', 4.841, '5.229,9', '2,53', 4.841, 322.677, 304.817)


In [8]:
# Cerrar la conexión a la base de datos
conn.close()

## Obtención de los datos cada minuto
Utilizamos la librería shedule para que el script se corra cada un minuto e ir obteniendo los valores actualizados. 

Primero creamos la función web_scraping() y copiamos el script utilizado. Esto nos servirá para posteriormente ejecutarlo cada un minuto con schedule.

In [7]:
#Crear la función para utilizar en schedule
def web_scraping():

    #Opciones de navegacion
    options =  webdriver.ChromeOptions()
    options.add_argument('--start-maximized')
    options.add_argument('--disable-extensions')

    driver_path = Service("C:\\Users\\Tatu\\Downloads\\chromedriver_win32\\chromedriver.exe")

    driver = webdriver.Chrome(service = driver_path, options = options)

    # Conexion con la pagina
    driver.get('https://es.investing.com/equities/ypf-sociedad')

    # Dejar que cargue la página
    time.sleep(5)

    tit=driver.title
    print(tit)

    # Insertar fecha y hora actual
    now = datetime.now()
    print(now)

    # Obtener los datos con el fullXPath

    ultimo_cierre = driver.find_element(By.XPATH, '/html/body/div/div[2]/div/div/div[2]/main/div/div[8]/div/div[2]/dl/div[1]/dd/span/span[1]')
    ultimo_cierre = ultimo_cierre.text
    print("Último cierre " + ultimo_cierre)

    rango_min = driver.find_element(By.XPATH, '/html/body/div/div[2]/div/div/div[2]/main/div/div[8]/div/div[2]/dl/div[2]/dd/span[1]/span[1]')
    rango_min = rango_min.text
    print("Mínimo " + rango_min)

    rango_max = driver.find_element(By.XPATH, '/html/body/div/div[2]/div/div/div[2]/main/div/div[8]/div/div[2]/dl/div[2]/dd/span[3]/span[1]')
    rango_max = rango_max.text
    print("Máximo " + rango_max)

    ingresos_T = driver.find_element(By.XPATH, '/html/body/div/div[2]/div/div/div[2]/main/div/div[8]/div/div[2]/dl/div[3]/dd/span/span[1]')
    ingresos_T = ingresos_T.text
    print("Ingresos " + ingresos_T + "T")

    apertura = driver.find_element(By.XPATH, '/html/body/div/div[2]/div/div/div[2]/main/div/div[8]/div/div[2]/dl/div[4]/dd/span/span[1]')
    apertura = apertura.text
    print("Apertura " + apertura)


    volumen = driver.find_element(By.XPATH, '/html/body/div/div[2]/div/div/div[2]/main/div/div[8]/div/div[2]/dl/div[7]/dd/span/span[1]')
    volumen = volumen.text
    print("Volumen " + volumen)

    volumen_promedio = driver.find_element(By.XPATH, '/html/body/div/div[2]/div/div/div[2]/main/div/div[8]/div/div[2]/dl/div[10]/dd/span/span[1]')
    volumen_promedio = volumen_promedio.text
    print("Volumen promedio " + volumen_promedio)   

    ultimo_precio = driver.find_element(By.XPATH, '/html/body/div/div[2]/div/div/div[2]/main/div/div[1]/div[2]/div[1]/span')
    ultimo_precio = ultimo_precio.text
    print("Último precio " + ultimo_precio)
    
    # Cerrar el driver
    driver.close()

    # Conexion con la base de datos
    conn = sqlite3.connect('acciones_YPFD.db')
    cursor = conn.cursor()

    # Insertar datos en la tabla
    cursor.execute("INSERT INTO resumen (date_time, ultimo_cierre, minimo, maximo, ingresos, apertura, volumen, volumen_promedio, ultimo_precio) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)", 
               (now, ultimo_cierre, rango_min, rango_max, ingresos_T, apertura, volumen, volumen_promedio, ultimo_precio))

    # Guardar los cambios en la base de datos y cerrar la conexión
    conn.commit()

    # Ver los datos guardados
    # Seleccionar los datos de la tabla
    cursor.execute("SELECT * FROM resumen")

    # Obtener los resultados de la consulta
    resultados = cursor.fetchall()

    # Imprimir los resultados en la consola
    for fila in resultados:
        print(fila)

    # Cerrar la conexión a la base de datos
    conn.close()



### Programar la ejecución cada minuto


In [10]:
# Programar la ejecución del scraping cada minuto
schedule.every().minute.do(web_scraping)

# Ejecutar el programa continuamente
while True:
    schedule.run_pending()
    time.sleep(1)

Acciones YPF | Cotización BA:YPFD hoy - Investing.com
2023-05-09 14:25:03.450658


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/div/div[2]/div/div/div[2]/main/div/div[8]/div/div[2]/dl/div[1]/dd/span/span[1]"}
  (Session info: chrome=112.0.5615.140)
Stacktrace:
Backtrace:
	GetHandleVerifier [0x0114DCE3+50899]
	(No symbol) [0x010DE111]
	(No symbol) [0x00FE5588]
	(No symbol) [0x010108F9]
	(No symbol) [0x01010AFB]
	(No symbol) [0x0103F902]
	(No symbol) [0x0102B944]
	(No symbol) [0x0103E01C]
	(No symbol) [0x0102B6F6]
	(No symbol) [0x01007708]
	(No symbol) [0x0100886D]
	GetHandleVerifier [0x013B3EAE+2566302]
	GetHandleVerifier [0x013E92B1+2784417]
	GetHandleVerifier [0x013E327C+2759788]
	GetHandleVerifier [0x011E5740+672048]
	(No symbol) [0x010E8872]
	(No symbol) [0x010E41C8]
	(No symbol) [0x010E42AB]
	(No symbol) [0x010D71B7]
	BaseThreadInitThunk [0x771A7D49+25]
	RtlInitializeExceptionChain [0x7792B74B+107]
	RtlClearBits [0x7792B6CF+191]


## Guardar datos en archivo CSV

In [11]:
# Abrir conexión a la base de datos sqlite
conn = sqlite3.connect('acciones_YPFD.db')

# Seleccionar los datos
cursor = conn.cursor()
cursor.execute("SELECT * FROM resumen")

# Crear archivo CSV y escribir datos
with open('resumen_YPF.csv', 'w', newline='') as csvfile:
    csvwriter = csv.writer(csvfile, delimiter=',')
    # Escribir encabezados de columnas
    csvwriter.writerow([i[0] for i in cursor.description])  
    csvwriter.writerows(cursor)

# Cerrar conexión a la base de datos
conn.close()
